In [17]:
import pathlib
import pymupdf4llm
import pymupdf.layout
import json

In [18]:
# md_text = pymupdf4llm.to_markdown('../data/recipes.pdf')
# pathlib.Path("../data/parsed_recipes.md").write_bytes(md_text.encode())

In [19]:
with open("../data/parsed_recipes.md", "r", encoding="utf-8") as f:
    lines = f.readlines()

In [20]:
chunks = {}
chunk_id = 0
content = ''
pages = []
for line in lines[199:]:
    stripped_line = line.replace("\n", " ")
    if '###' in stripped_line:
        chunks[str(chunk_id)] = {'content':content, 'metadatas':{'pages': pages}}
        chunk_id += 1
        content = ''
        pages = []
    if stripped_line.strip().isdigit():
        pages.append(stripped_line.strip())
        continue
    content += stripped_line

In [21]:
with open('../data/chunks.json', 'w', encoding='utf-8') as f:
    json.dump(chunks, f, indent=4)

In [22]:
import chromadb

In [23]:
chroma_client = chromadb.Client()

In [24]:
collection = chroma_client.create_collection(name="recipes_collection")

In [25]:
chunks.__delitem__('0')

In [26]:
for chunk_id, chunk in chunks.items():
    collection.add(
        ids = chunk_id,
        documents=chunk['content'],
        metadatas=chunk['metadatas'],
    )

In [27]:
results = collection.query(
    query_texts=["healthy meal"],
    n_results=2
)

In [28]:
from langchain_ollama import ChatOllama
from langchain.tools import tool

In [29]:
model = ChatOllama(model='qwen3:4b')

In [30]:
@tool(response_format='content_and_artifact')
def retrieve_context(query: str):
    """
    Retrieve information to help answer a query.
    """
    results = collection.query(
        query_texts=[f"{query}"],
        n_results=1
    )
    retrieved_docs = results['documents']
    serialized = "\n\n".join(
        (f"Source: {results['metadatas'][i]}\nContent: {retrieved_docs[i]}")
        for i in range(len(retrieved_docs))
    )

    return serialized, results

In [31]:
from langchain.agents import create_agent

In [32]:
tools = [retrieve_context]
agent = create_agent(model=model, tools=tools)

In [33]:
query=("what healthy meal should i eat today?\n\n"
       "Once you find the answer, give me clear ingredients, instructions and nutritional values")

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values"
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

what healthy meal should i eat today?

Once you find the answer, give me clear ingredients, instructions and nutritional values
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (c684aeab-799a-42c0-94da-6a0c9a933147)
 Call ID: c684aeab-799a-42c0-94da-6a0c9a933147
  Args:
    query: what healthy meal should i eat today?
================================= Tool Message =================================
Name: retrieve_context

Source: [{'pages': ['35', '36']}]
Content: ['###### **SWEET** ## POTATO STEW  Ingredients 1 large onion, thinly sliced 3 garlic cloves, minced 1 tbsp fresh ginger, minced 2 tbsp red wine or 2 tbsp cooking sherry 1 tsp cumin 1 tsp cinnamon 1 tsp paprika ½ tsp crushed red pepper flakes, to taste 1 C. water ½ tsp salt  ¼ C. diced dried apricot 2 C. cooked chickpeas ¼ C. raisins 2 tbsp lemon juice ¼ C. sliced almonds, toasted in a